# 주문 정보 생성
- ## input : 생성할 주문의 개수(default = 1000)
#
- ## output : 주문 데이터가 저장된 엑셀 파일

In [1]:
import pandas as pd
import numpy as np
import random

class ORDER:
    
    def __init__(self,row = 1000):   
        columns=['플랜트', '플랜트명', '매장코드', '매장', '상품코드', '상품명', '차수', '주문일자','운송장번호',
                '운송장출력여부', '주문취소여부', '주문번호', '받는이', '우편번호', '주소1', '주소2', '전화번호1',
                '전화번호2', '배송기재사항', '판매가', '합포여부', '예정수량', '검수수량', '출력일시', '출력자', '몰구분',
                'DAS여부', 'DAS차수', 'DAS지시번호', 'BOXNO', 'CELLNO', '추가박스여부']
        dependent_key = ['플랜트', '상품코드']
        
        # 주문 데이터를 저장할 데이터 구조
        self.order = pd.DataFrame([],columns=columns)
        
        # 상관관계 없는 칼럼들의 셋을 만든다.
        self.Independent_set = {}
        for col in columns :
            try :
                f = open('Sample/'+col,'r',encoding = 'utf-8')
            except :
                f = open('Sample/'+col,'r',encoding = 'cp949')
            data = f.read()[:-1].split('|')
            self.Independent_set[col] = data
            
        # 상관관계가 존재하는 칼럼들의 셋을 만든다.
        self.Dependent_set = {}
        for i in range(1,3) :         
            try :
                f = open('Sample/relation'+str(i) ,'r',encoding = 'utf-8')
            except :
                f = open('Sample/relation'+str(i) ,'r',encoding = 'cp949')
            data = f.read()[:-1].split('\n')
            data = [x.split('|') for x in data]
            self.Dependent_set[dependent_key[i-1]] = data
        
        # row의 개수만큼 주문 데이터를 만든다.
        for _ in range(row):
            new_random_data = []
            for col in dependent_key:
                new_random_data.extend(self.__choice_dependent(col))                
                    
            for col in columns[6:]:
                new_random_data.append(self.__choice_independent(col))
            self.order.loc[len(self.order)] = new_random_data
    
    #independent_set 에서 choice하는 함수.(private)
    #INPUT      - (str) column
    #OUTPUT     - (str) 포아송 함수로 chice 되어진 value
    def __choice_independent(self,name) :
        if name == '주문취소여부' or name == '추가박스여부' :                                         
            return random.choices(['Y', 'N'], weights = (5, 95))[0]
        elif name == '운송장출력여부':
            return random.choices(['Y', 'N'], weights = (95, 5))[0]
        else:
            rand_pois = np.random.poisson(lam=10, size=len(self.Independent_set[name]))
            return random.choices(self.Independent_set[name], weights = rand_pois)[0]
    
    #dependent_set 에서 choice하는 함수.(private)
    #INPUT      - (str) column.
    #OUTPUT     - (str) 포아송 함수로 chice 되어진 value.
    def __choice_dependent(self,name):
        rand_pois = np.random.poisson(lam=10, size=len(self.Dependent_set[name]))
        # print(rand_pois)
        return random.choices(self.Dependent_set[name], weights = rand_pois)[0]
    
    #주문 내역 출력함수.
    #INPUT      - none.
    #OUTPUT     - (판다스 DataFrame) 주문내역.
    def show(self):
        return self.order
    
    #주문 내역 excel 변환 함수.
    #INPUT      - (str) excel 이름.
    def to_excel(self,name = '주문 정보'):
        self.order.to_excel(name+'.xlsx', encoding='cp949')                                    #OUTPUT     - (.xlsx) excel 파일.
        print(f'주문 정보 파일 --- {name}.xlsx 이(가) 생성 되었습니다.')

# 상품 정보 생성
- ## input : 주문 데이터의 경로와 파일명, 저장할 csv 파일의 파일명(default = 상품 정보)
#
- ## output : 상품 데이터가 저장된 csv 파일

In [2]:
def random_product_creator(in_name, out_name = '상품 정보') :
    in_name += '.xlsx'
    df=pd.read_excel(in_name)
    df = pd.DataFrame({
        '상품코드':df["상품코드"],
        '상품명':df['상품명'],
        'ABC': [['A', 'B', 'C'][np.random.randint(0, 3)] for i in range(len(df))], 
        '단품가로(mm)':  [np.random.randint(1,100) for i in range(len(df))], 
        '단품세로(mm)': [np.random.randint(1,100) for i in range(len(df))], 
        '단품높이(mm)':  [np.random.randint(1,100) for i in range(len(df))], 
        '중량(g)':  [np.random.randint(1, 300) for i in range(len(df))]   
    })
    df.to_csv(out_name+'.csv', index=False, encoding='cp949')
    print(f'상품 정보 파일 --- {out_name}.csv 이(가) 생성 되었습니다.')


# 맵 자료구조 생성
- ## input : 공장의 행 개수(default=1), 공장의 열 개수(default=1), 선반의 개수(default = 987654321)
#
- ##         생성되는 공장의 크기는 (10 + 행x10) x (열x8) 이다.
#
- ## output : 상품을 적재, 추출하고 적재된 상품의 정보를 확인할 수 있는 자료구조
      

In [3]:

class SHELF :
    def __init__(self) :
        self.products = []
    
    # 선반에 상품을 적재함
    def push(self, product) :
        self.products.append(product)
        
    # 선반에 상품을 추출함 (선반에 해당 상품이 2개 이상 있는 경우 가장 먼저 들어온 상품을 추출함)
    def pop(self, product) :
        for i in range(len(self.products)) :
            if self.products[i].code == product.code :
                del self.products[i]
                return
    # 선반에 물건이 몇개 들어있는 지 알려준다.
    def stuffed(self) :
        return len(self.products)
    
    #데이터 보여주기용 함수
    def show(self) :
        if len(self.products) == 0 :
            print("EMPTY")
        for product in self.products :
            print(product)
        print()
        
        
    
class PRODUCT :
    #  현재 단계에서 잘모르는 데이터에 대해서는 defalut 값으로 처리함
    def __init__(self, code, name = '알수없음', ABC = '.', x = '.', y = '.', z = '.', g = '.') :
        self.code = code
        self.name = name
        self.ABC = ABC
        self.x = x
        self.y = y
        self.z = z
        self.g = g
    def __repr__(self) :
        return f"code = {self.code}, name = {self.name}"
        

class MAP :
    # 워크스테이션 당 맡게 되는 선반의 개수를 일관되게 유지하기 위해서
    # 가로, 세로가 n,m 으로 주어진 경우 공장 크기는 n*8, 10 + m*10 으로 생성됨

    def __init__(self,R=1,C=1,K=987654321) :
        if R < 1 or C < 1: 
            raise ValueError("공장 크기가 적절하지 않습니다.\n")
        R *= 10
        R += 10
        C *= 8
        self.R = R
        self.C = C
        self.K = K
        self.index = 1
        self.map = [[0]*C for _ in range(R)]
        self.products = {}
        
        # 워크스테이션 설정
        for c in range(3,C,8) :
            self.map[3][c] = -1
        for c in range(7,C-4,8) :
            self.map[R-4][c] = -1
            
        # 선반 설정
        break_point = False
        for c in range(1,self.C-2,4) :
            if break_point : break
            for r in range(7,self.R-10,10) :
                if break_point : break
                for dc in range(2) :
                    if break_point : break
                    for dr in range(6) :
                        cr = r + dr
                        cc = c + dc 
                        self.map[cr][cc] = self.index
                        self.index += 1
                        if self.index > K :
                            break_point = True
                            break
        
        # 선반 배열 생성
        self.shelves = [SHELF() for _ in range(self.index)]
            
    # 해당 인덱스를 가진 선반에 물건을 넣는다.
    def push(self, product, index = 0, show = False) :
        # 인덱스가 변수로 주어져 있지 않다면 비어있는 선반을 찾아서 적재한다.
        if index == 0 :
            Full = True
            for i in range(1,self.index) :
                if self.shelves[i].stuffed() < 60 :
                    index = i
                    Full = False
                    break
            if Full :
                print("모든 선반에 물건이 차있기 때문에 적재할 수 없습니다.\n")
                return
            
        # 해당 선반에 물건이 60개 이상이라면 적재하지 않는다.
        if self.shelves[index].stuffed() >= 60 :
            print("f{index} 번 선반이 가득 차 적재할 수 없습니다.")
            return

        # 해당 선반에 물품을 적재한다. 
        self.shelves[index].push(product)
        # 적재된 상품이 몇번째 선반에 있는지 저장한다.
        if product.code in self.products.keys() :
            self.products[product.code][index] += 1
            self.products[product.code][0] += 1
        else :
            self.products[product.code] = [0 for _ in range(self.index)]
            self.products[product.code][index] += 1
            self.products[product.code][0] += 1
        
        if show :
            print(f"{product.name} 이 {index} 번째 선반에 적재되었습니다.\n")
            
    # 해당 인덱스를 가진 선반에서 물건을 뺀다.
    def pop(self, product, index = 0, show = False) :
        # 상품이 공장에 존재하지 않는 경우
        if product.code not in self.products.keys():
            print(f"{product.name} 이 공장에 존재하지 않습니다.\n")
            return
        
        # 상품이 공장에 존재하나 인덱스가 주어지지 않은 경우 자동으로 인덱스를 구함
        if index == 0 :
            for i,cnt in enumerate(self.products[product.code][1:]) :
                if cnt > 0 :
                    index = i+1
                    break
        
        # 상품이 공장에 존재하고 인덱스가 주어진 경우인데도 해당 선반에 상품이 존재하지 않은 경우
        if self.products[product.code][index] == 0 :
            print(f"{product.name} 이 해당 선반에 존재하지 않습니다.")
            return
        
        # 상품을 추출함
        if show :
            print(f"{product.name} 이 {index} 번째 선반에 추출되었습니다.\n")
        self.products[product.code][index] -= 1
        self.products[product.code][0] -= 1
        if self.products[product.code][0] == 0 :
            del self.products[product.code]
        self.shelves[index].pop(product)
        
    # 인풋 : 상품 데이터가 저장된 csv 파일의 경로와 이름
    # 상품 데이터를 불러와 맵에 적재해 줌
    def push_orders(self, name, head = 0, show = False) :
        name += '.csv'
        order_data = pd.read_csv(name, encoding = 'cp949')
        if head > 0 :
            order_data = order_data[:head]
        columns = ['상품코드', '상품명', 'ABC', '단품가로(mm)', '단품세로(mm)', '단품높이(mm)','중량(g)']
        for i in range(len(order_data)) :
            product = PRODUCT(order_data.loc[i][columns[0]], order_data.loc[i][columns[1]], order_data.loc[i][columns[2]], 
                            order_data.loc[i][columns[3]], order_data.loc[i][columns[4]], order_data.loc[i][columns[5]], 
                            order_data.loc[i][columns[6]])
            self.push(product, show = show)
    
    # 인풋 : 상품 데이터가 저장된 csv 파일의 경로와 이름
    # 상품 데이터를 풀러와 맵에서 추출해 줌
    def pop_orders(self, name, head = 0, show = False) :
        name += '.csv'
        order_data = pd.read_csv(name, encoding = 'cp949')
        if head > 0 :
            order_data = order_data[:head]
        columns = ['상품코드', '상품명', 'ABC', '단품가로(mm)', '단품세로(mm)', '단품높이(mm)','중량(g)']
        for i in range(len(order_data)) :
            product = PRODUCT(order_data.loc[i][columns[0]], order_data.loc[i][columns[1]], order_data.loc[i][columns[2]], 
                            order_data.loc[i][columns[3]], order_data.loc[i][columns[4]], order_data.loc[i][columns[5]], 
                            order_data.loc[i][columns[6]])
            self.pop(product, show = show)
            
    # 상품이 몇번 선반에 들어있는지 알려준다.
    def show_product(self, product) :
        # 상품이 공장에 적재되어있지 않는 경우
        if product.code not in self.products.keys():
            print(f"{product.name} 이 공장에 존재하지 않습니다.\n")
            return
        
        print(product, f", 총 {self.products[product.code][0]} 개")
        for index,cnt in enumerate(self.products[product.code][1:]) :
            if cnt == 0 :
                continue
            print(f"{index+1}번 선반 - {cnt} 개")
        print()
    
             
    # 해당 인덱스를 가진 선반이 어떤 물건을 가지고 있는지 보여준다.
    def show_shelf(self, index) :
        print(f"{index} 번째 선반에 들어있는 상품 : ")
        self.shelves[index].show()

    
    # MAP의 형태를 보여준다. W = workstation, @ = 선반
    # index = True 일 때는 선반의 인덱스를 보여준다. 1,2,3... = 선반의 index, -1 = workstation
    def show(self, index = False) :
        print(f"R : {self.R} C : {self.C}\n")
        
        if index :
            for r in range(self.R) :
                for c in range(self.C) :
                    print(str(self.map[r][c]).ljust(len(str(self.index))), end = ' ')
                print()
            return
        for r in range(self.R) :
            for c in range(self.C) :
                if self.map[r][c] == -1 :
                    print('W', end = ' ')
                elif self.map[r][c] > 0 :
                    print("@", end = ' ')
                else : print('-', end = ' ')
            print("\n")


# 사용예시

# 주문 정보 생성

In [4]:
# 주문 정보를 담고 있는 클래스를 생성한다.
random_order = ORDER(100)
# 클래스가 담고 있는 주문 정보를 엑셀로 만든다.
random_order.to_excel('주문 정보')

주문 정보 파일 --- 주문 정보.xlsx 이(가) 생성 되었습니다.


# 상품 정보 생성

In [5]:
# 주문 정보로 부터 상품 정보를 생성한다.
# 상품 정보가 저장된 'csv' 파일을 만든다.
random_product_creator(in_name='주문 정보', out_name='상품 정보')

상품 정보 파일 --- 상품 정보.csv 이(가) 생성 되었습니다.


# 맵 생성

In [6]:
# 맵 생성
factory = MAP(1,1)

In [7]:
# 맵 구조 확인
# 선반 = '@', 워크스테이션 = 'W'
factory.show()

R : 20 C : 8

- - - - - - - - 

- - - - - - - - 

- - - - - - - - 

- - - W - - - - 

- - - - - - - - 

- - - - - - - - 

- - - - - - - - 

- @ @ - - @ @ - 

- @ @ - - @ @ - 

- @ @ - - @ @ - 

- @ @ - - @ @ - 

- @ @ - - @ @ - 

- @ @ - - @ @ - 

- - - - - - - - 

- - - - - - - - 

- - - - - - - - 

- - - - - - - - 

- - - - - - - - 

- - - - - - - - 

- - - - - - - - 



In [8]:
#맵 구조 확인
# 1 이상의 자연수 : 선반의 인덱스, -1 = 워크스테이션
factory.show(index = True)

R : 20 C : 8

0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
0  0  0  -1 0  0  0  0  
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
0  1  7  0  0  13 19 0  
0  2  8  0  0  14 20 0  
0  3  9  0  0  15 21 0  
0  4  10 0  0  16 22 0  
0  5  11 0  0  17 23 0  
0  6  12 0  0  18 24 0  
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  


In [9]:
# 상품 적재
# 상품을 구분하는 기준은 오직 "상품 코드" 하나이다.
# 따라서 상품코드만 다르고 나머지 특성은 전부 같더라도 같은 상품으로 구분한다.
# 반대로 상품코드만 정확히 알고 있다면 상품을 적재, 추출할 수 있다.
factory.push_orders('상품 정보', show = True)

레알 나비 막대봉 (루비) 이 1 번째 선반에 적재되었습니다.

스파클 토트 CREAM 이 1 번째 선반에 적재되었습니다.

트랩워터 모어라이크 에센스 앤드 쿨리 이 1 번째 선반에 적재되었습니다.

동물마을 고래 이 1 번째 선반에 적재되었습니다.

아티바바 말랑이 세트 이 1 번째 선반에 적재되었습니다.

윈드레이서 스프리트 이 1 번째 선반에 적재되었습니다.

부드와 신규 쇼핑백 이 1 번째 선반에 적재되었습니다.

로티새들크로스바디 TRUE NAVY XXX 이 1 번째 선반에 적재되었습니다.

소피아 토트 VANILLA CREAM 이 1 번째 선반에 적재되었습니다.

포인트앤뷰 스탠바이미  BLC 27인치 이 1 번째 선반에 적재되었습니다.

스텔라 숄더_SAND BROWN 이 1 번째 선반에 적재되었습니다.

크루니슬리퍼 CARAMEL 235 이 1 번째 선반에 적재되었습니다.

페인트컵캐디_5 이 1 번째 선반에 적재되었습니다.

제이미 N 쇼퍼BLACK 이 1 번째 선반에 적재되었습니다.

캡슐커피: 일리 네스프레소 호환캡슐 포르테 이 1 번째 선반에 적재되었습니다.

칼라붓_노랑 이 1 번째 선반에 적재되었습니다.

Rainbow socks 6pack for baby 이 1 번째 선반에 적재되었습니다.

우유팩딸기 이 1 번째 선반에 적재되었습니다.

루키크로스바디 TOFFEE TAN 이 1 번째 선반에 적재되었습니다.

포인트앤뷰 거치형  BLC 24W9 이 1 번째 선반에 적재되었습니다.

길라 롱웨어 립크림 12호 이 1 번째 선반에 적재되었습니다.

오웬 크로스바디_BLACK 이 1 번째 선반에 적재되었습니다.

고무롤러_노이즈 이 1 번째 선반에 적재되었습니다.

로티새들크로스바디 DUSTY MAUVE XXX 이 1 번째 선반에 적재되었습니다.

셀레나 버킷 BLACK 이 1 번째 선반에 적재되었습니다.

돔-글 블러셔 #CR01 진저코랄 이 1 번째 선반에 적재되었습니다.

코니 쇼퍼_BLACK 이 1 번째 선반에 적재되었습니다.

동

In [10]:
# 적재된 상품 확인
# 1번 선반에 적재된 상품 확인
factory.show_shelf(1)

1 번째 선반에 들어있는 상품 : 
code = CMZ110212, name = 레알 나비 막대봉 (루비)
code = O1FBTT24004XXX, name = 스파클 토트 CREAM
code = 8809793590055, name = 트랩워터 모어라이크 에센스 앤드 쿨리
code = CMZ110403, name = 동물마을 고래
code = AB_SQ, name = 아티바바 말랑이 세트
code = RM204S007WH235, name = 윈드레이서 스프리트
code = 0056-01, name = 부드와 신규 쇼핑백
code = O9SBCB81127XXX, name = 로티새들크로스바디 TRUE NAVY XXX
code = O1SBTT42130XXX, name = 소피아 토트 VANILLA CREAM
code = 72SQ-021133, name = 포인트앤뷰 스탠바이미  BLC 27인치
code = O2SBSD61133XXX, name = 스텔라 숄더_SAND BROWN
code = O9SSSR78019235, name = 크루니슬리퍼 CARAMEL 235
code = AB_PCAD_5, name = 페인트컵캐디_5
code = O0FBSP38009XXX, name = 제이미 N 쇼퍼BLACK
code = 8003753158709, name = 캡슐커피: 일리 네스프레소 호환캡슐 포르테
code = CBR-YL, name = 칼라붓_노랑
code = PP204P0406PK, name = Rainbow socks 6pack for baby
code = CMZ110731, name = 우유팩딸기
code = O0FBCB43118XXX, name = 루키크로스바디 TOFFEE TAN
code = 72SQ-021011, name = 포인트앤뷰 거치형  BLC 24W9
code = 8809491945485, name = 길라 롱웨어 립크림 12호
code = O2SBCB64009XXX, name = 오웬 크로스바디_BLACK
code = 900078, name = 

In [11]:
# 적재된 상품 확인
# 귀엽냥이 적재된 선반의 위치 확인
factory.show_product(PRODUCT('CMZ111264', '귀엽냥'))

귀엽냥 이 공장에 존재하지 않습니다.



In [12]:
# 상품 추출
factory.pop_orders('상품 정보', show = True)

레알 나비 막대봉 (루비) 이 1 번째 선반에 추출되었습니다.

스파클 토트 CREAM 이 1 번째 선반에 추출되었습니다.

트랩워터 모어라이크 에센스 앤드 쿨리 이 1 번째 선반에 추출되었습니다.

동물마을 고래 이 1 번째 선반에 추출되었습니다.

아티바바 말랑이 세트 이 1 번째 선반에 추출되었습니다.

윈드레이서 스프리트 이 1 번째 선반에 추출되었습니다.

부드와 신규 쇼핑백 이 1 번째 선반에 추출되었습니다.

로티새들크로스바디 TRUE NAVY XXX 이 1 번째 선반에 추출되었습니다.

소피아 토트 VANILLA CREAM 이 1 번째 선반에 추출되었습니다.

포인트앤뷰 스탠바이미  BLC 27인치 이 1 번째 선반에 추출되었습니다.

스텔라 숄더_SAND BROWN 이 1 번째 선반에 추출되었습니다.

크루니슬리퍼 CARAMEL 235 이 1 번째 선반에 추출되었습니다.

페인트컵캐디_5 이 1 번째 선반에 추출되었습니다.

제이미 N 쇼퍼BLACK 이 1 번째 선반에 추출되었습니다.

캡슐커피: 일리 네스프레소 호환캡슐 포르테 이 1 번째 선반에 추출되었습니다.

칼라붓_노랑 이 1 번째 선반에 추출되었습니다.

Rainbow socks 6pack for baby 이 1 번째 선반에 추출되었습니다.

우유팩딸기 이 1 번째 선반에 추출되었습니다.

루키크로스바디 TOFFEE TAN 이 1 번째 선반에 추출되었습니다.

포인트앤뷰 거치형  BLC 24W9 이 1 번째 선반에 추출되었습니다.

길라 롱웨어 립크림 12호 이 1 번째 선반에 추출되었습니다.

오웬 크로스바디_BLACK 이 1 번째 선반에 추출되었습니다.

고무롤러_노이즈 이 1 번째 선반에 추출되었습니다.

로티새들크로스바디 DUSTY MAUVE XXX 이 1 번째 선반에 추출되었습니다.

셀레나 버킷 BLACK 이 1 번째 선반에 추출되었습니다.

돔-글 블러셔 #CR01 진저코랄 이 1 번째 선반에 추출되었습니다.

코니 쇼퍼_BLACK 이 1 번째 선반에 추출되었습니다.

동

In [13]:
# 상품이 공장에서 추출되었는지 확인
factory.show_shelf(1)

1 번째 선반에 들어있는 상품 : 
EMPTY



In [14]:
# 귀엽냥이 추출되었는지 확인
factory.show_product(PRODUCT('CMZ111264', '귀엽냥'))

귀엽냥 이 공장에 존재하지 않습니다.

